<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/main/Language_Modelling%5BMLM%2BCLM%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn
!pip install accelerate

In [2]:
%%bash
nvidia-smi
python --version
which nvcc
echo $PATH
echo $LD_LIBRARY_PATH


Fri Jul 29 15:25:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
import transformers,torch,accelerate
transformers.__version__,torch.__version__,accelerate.__version__

('4.21.0', '1.12.0+cu113', '0.11.0')

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM
import torch
model_name = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [47]:
model_mlm = AutoModelForMaskedLM.from_pretrained(model_name)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

In [6]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## Pre processing the data

In [7]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
imdb_dataset['train'][0]

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are f

Tokenize the whole corpus and add one key containing the word_ids of the tokens

In [9]:
def tokenize_text(batch_of_data):
  batch_encoding = tokenizer(batch_of_data['text'])
  word_ids = []
  for idx,each_input_id in enumerate(batch_encoding['input_ids']):
    word_ids.append(batch_encoding.word_ids(idx))
  batch_encoding['word_ids'] = word_ids
  return batch_encoding

In [10]:
tokenized_datasets = imdb_dataset.map(
    tokenize_text, batched=True, remove_columns=["text", "label"]
)

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets,len(tokenized_datasets['train'][0]['word_ids']),len(tokenized_datasets['train'][0]['input_ids']),len(tokenized_datasets['train']['word_ids'])

(DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
}),
 363,
 363,
 25000)

Now, concatenate all the tokens and then chunk it to a context_length. Here, I have taken context_length of 128 tokens considering GPU memory of google colab

In [12]:
# let's take 3 sample and print out the number of tokens per review:
sample = tokenized_datasets['train'][:3]
for idx,each_sample in enumerate(sample['input_ids']):
  print(f"review {idx} has {len(each_sample)} tokens")
sample.keys(),

review 0 has 363 tokens
review 1 has 304 tokens
review 2 has 133 tokens


(dict_keys(['input_ids', 'attention_mask', 'word_ids']),)

In [13]:
concatenated_example ={}
for key in sample.keys():
  concatenated_example[key] = sum(sample[key],[])


In [14]:
len(sum(sample['input_ids'],[])),len(concatenated_example['input_ids'])

(800, 800)

In [15]:
[_ for _ in range(0,20,5)][:3]

[0, 5, 10]

In [16]:
# now let's chunk it to 128
chunked_sample ={}
chunk_size = 128
for i in range(0,len(concatenated_example['input_ids']),chunk_size):
  print(i,i+chunk_size)


0 128
128 256
256 384
384 512
512 640
640 768
768 896


In [17]:
temp_list = []
for k,v in concatenated_example.items():
  for i in range(0,len(concatenated_example['input_ids']),chunk_size):
    temp_list.append(v[i:i+chunk_size])
  chunked_sample[k] = temp_list
  temp_list = []

In [18]:
chunked_sample.keys(),len(chunked_sample['input_ids']),[len(_) for _ in chunked_sample['input_ids']]

(dict_keys(['input_ids', 'attention_mask', 'word_ids']),
 7,
 [128, 128, 128, 128, 128, 128, 32])

as we are seeing the last chunk is smaller than the chunk size. There are two options for this : either to drop the last chunk or to pad to the chunk size. I will drop the last chuk here.

In [19]:
800/128,(800//128)*128,800//128

(6.25, 768, 6)

In [20]:
chunk_size = 128
def concatenate_and_chunk(batch_of_data):
  concatenated_data = {}
  for key in batch_of_data.keys():
    concatenated_data[key] = sum(batch_of_data[key],[])
  total_length = len(concatenated_data['input_ids'])
  total_length = (total_length//chunk_size)*chunk_size # modify the total length to drop the last chunk
  chunked_data = {}
  for k,v in concatenated_data.items():
    temp_list = []
    for i in range(0,total_length,chunk_size):
      temp_list.append(v[i:i+chunk_size])
    chunked_data[k] = temp_list
  chunked_data ['labels'] = chunked_data['input_ids'].copy()
  return chunked_data

  
  

In [21]:
sample_data = tokenized_datasets['train'].shuffle(seed=42).select(range(3))

In [22]:
chunked_sample_data = sample_data.map(concatenate_and_chunk,batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
[len(_) for _ in sample_data['input_ids']],[len(_) for _ in chunked_sample_data['input_ids']],[len(_) for _ in chunked_sample_data['labels']],

([179, 159, 237], [128, 128, 128, 128], [128, 128, 128, 128])

In [24]:
lm_datasets = tokenized_datasets.map(concatenate_and_chunk,batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [25]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

Now one final thing is left, that is inserting [MASK] tokens at random positions in the inputs. Let's do that now

In [26]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm_probability=0.15)

In [27]:
"""data collator is basically a collation function , which will take list of samples aka list of dict and form the batch, 
which dataloaded can use during training"""
some_samples = [lm_datasets['train'][i] for i in range(2)]
for _ in some_samples:
  _.pop('word_ids')
  print(tokenizer.decode(_['input_ids'],))


[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such
as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes a

In [28]:
len(lm_datasets['train']['input_ids'])

61291

In [29]:
collated_samples = data_collator(some_samples)
collated_samples

{'input_ids': tensor([[  101,  1045,   103,  1045,  2572,  8025,  1011,  3756,   103,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,   103,  2008,  5129,  2009,
          2043,  2009,  2001,  2034,  2207,   103,  3476,   103,  1045,  2036,
          2657,  2008,  2012,   103,  2009,  2001,  8243,  2011,  1057,  1012,
           103,   103,  8205,  2065,  2009,  2412,  2699,   103,  4607,  2023,
          2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
           103,  1000,  1045,  2428,   103,  2000,  2156,  2023,  2005,  2870,
          1012,  1026,  7987,  1013,   103,   103,  7987,  1013,  1028,  1996,
          5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,   103,
         14229,  2040,  4122,  2000,  4553,  2673,  2016,   103,  2055,  2166,
          1012,  1999,  3327,  2016,  4122,  2000,   103,  2014, 26819,  2015,
          2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
         25430, 14728,   103,  2055,  

In [30]:
collated_samples['input_ids'].shape

torch.Size([2, 128])

In [31]:
for _ in collated_samples['input_ids']:
  print(tokenizer.decode(_))

[CLS] i [MASK] i am curious - yellow [MASK] my video store because of all the [MASK] that surrounded it when it was first released [MASK] 1967 [MASK] i also heard that at [MASK] it was seized by u. [MASK] [MASK] customs if it ever tried [MASK] enter this country, therefore being a fan of films considered " [MASK] " i really [MASK] to see this for myself. < br / [MASK] [MASK] br / > the plot is centered around a young swedish drama student [MASK] lena who wants to learn everything she [MASK] about life. in particular she wants to [MASK] her axials to making some sort of documentary on what the average swede [MASK] about certain political issues such
[MASK] the vietnam war and race issues in the united states. in between [MASK] politicians and ordinary denizens of stockholm about their [MASK] on politics, she has [MASK] with her [MASK] teacher, classmates, and married men. < br [MASK] > < br / > what kills me about i am curious - [MASK] is [MASK] 40 years ago, this was considered [MASK].

In [32]:
import collections
import numpy as np
from transformers import default_data_collator
wwm_probability = 0.2

In [33]:
some_samples = [lm_datasets['train'][i] for i in range(2)]
type(some_samples[0]['input_ids']),some_samples[0].keys()

(list, dict_keys(['input_ids', 'attention_mask', 'word_ids', 'labels']))

In [34]:
from transformers import DataCollatorForWholeWordMask
data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer)
collated_sample = data_collator(some_samples)

In [35]:
for _ in collated_samples['input_ids']:
  print(tokenizer.decode(_))

[CLS] i [MASK] i am curious - yellow [MASK] my video store because of all the [MASK] that surrounded it when it was first released [MASK] 1967 [MASK] i also heard that at [MASK] it was seized by u. [MASK] [MASK] customs if it ever tried [MASK] enter this country, therefore being a fan of films considered " [MASK] " i really [MASK] to see this for myself. < br / [MASK] [MASK] br / > the plot is centered around a young swedish drama student [MASK] lena who wants to learn everything she [MASK] about life. in particular she wants to [MASK] her axials to making some sort of documentary on what the average swede [MASK] about certain political issues such
[MASK] the vietnam war and race issues in the united states. in between [MASK] politicians and ordinary denizens of stockholm about their [MASK] on politics, she has [MASK] with her [MASK] teacher, classmates, and married men. < br [MASK] > < br / > what kills me about i am curious - [MASK] is [MASK] 40 years ago, this was considered [MASK].

##Fine Tuning


To train a model we need :
1. tokenized input and output 
2. torch.utils Dataset and Dataloader
3. Loss function, optimizer, learning rate scheduler
4. Evaluation Loop
5. Saving the best model a

In [36]:
# downsample the data
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [37]:
%%bash
apt install git-lfs

Reading package lists...
Building dependency tree...
Reading state information...
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [39]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [51]:
from transformers import TrainingArguments,Trainer
batch_size = 16
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_name.split("/")[-1]
train_args =TrainingArguments(output_dir=f"{model_name}-finetuned-imdb",
                              evaluation_strategy="epoch",per_device_train_batch_size=batch_size,
                              per_device_eval_batch_size=batch_size,gradient_accumulation_steps=4,
                              learning_rate=2e-5,weight_decay=0.01,lr_scheduler_type="linear",
                              logging_strategy="epoch",save_strategy="epoch",fp16=True,optim="adamw_hf",
                              push_to_hub=True,hub_strategy="all_checkpoints",)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
trainer = Trainer(model=model_mlm,args=train_args,data_collator=data_collator,
                  train_dataset=downsampled_dataset["train"],eval_dataset=downsampled_dataset["test"],
                  tokenizer=tokenizer,)

/content/distilbert-base-uncased-finetuned-imdb is already a clone of https://huggingface.co/susghosh/distilbert-base-uncased-finetuned-imdb. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [53]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 468


Epoch,Training Loss,Validation Loss
0,2.966700,2.779457
1,2.861200,2.691012
2,2.807500,2.704437


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-imdb/checkpoint-156
Configuration saved in distilbert-base-uncased-finetuned-imdb/checkpoint-156/config.json
Model weights saved in distilbert-base-uncased-finetuned-imdb/checkpoint-156/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-imdb/checkpoint-156/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-imdb/checkpoint-156/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-imdb/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-imdb/special_tokens_map.json
The following co

TrainOutput(global_step=468, training_loss=2.878475352230235, metrics={'train_runtime': 213.6251, 'train_samples_per_second': 140.433, 'train_steps_per_second': 2.191, 'total_flos': 993678426095616.0, 'train_loss': 2.878475352230235, 'epoch': 3.0})

In [54]:
eval_results = trainer.evaluate()
eval_results

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'epoch': 3.0,
 'eval_loss': 2.734124183654785,
 'eval_runtime': 2.1886,
 'eval_samples_per_second': 456.916,
 'eval_steps_per_second': 28.786}

In [56]:
import math
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 15.40


In [57]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-imdb
Configuration saved in distilbert-base-uncased-finetuned-imdb/config.json
Model weights saved in distilbert-base-uncased-finetuned-imdb/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-imdb/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-imdb/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file checkpoint-468/optimizer.pt:   0%|          | 3.34k/511M [00:00<?, ?B/s]

Upload file checkpoint-156/optimizer.pt:   0%|          | 3.34k/511M [00:00<?, ?B/s]

Upload file checkpoint-312/pytorch_model.bin:   0%|          | 3.34k/256M [00:00<?, ?B/s]

Upload file checkpoint-312/rng_state.pth:  24%|##3       | 3.34k/14.1k [00:00<?, ?B/s]

...

Upload file checkpoint-312/scaler.pt: 100%|##########| 559/559 [00:00<?, ?B/s]

Upload file checkpoint-468/scaler.pt: 100%|##########| 559/559 [00:00<?, ?B/s]

Upload file runs/Jul29_15-59-12_5b678e102dab/events.out.tfevents.1659110616.5b678e102dab.73.2: 100%|##########…

To https://huggingface.co/susghosh/distilbert-base-uncased-finetuned-imdb
   91ab64b..8e35042  main -> main

   91ab64b..8e35042  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}, 'dataset': {'name': 'imdb', 'type': 'imdb', 'config': 'plain_text', 'split': 'train', 'args': 'plain_text'}}
To https://huggingface.co/susghosh/distilbert-base-uncased-finetuned-imdb
   8e35042..0c30010  main -> main

   8e35042..0c30010  main -> main



'https://huggingface.co/susghosh/distilbert-base-uncased-finetuned-imdb/commit/8e350423b4192133276cc5a54e80f2d4e54ea034'